VERSIÓN ÚNICAMENTE CON DEEPFACE

In [ ]:
import cv2
from deepface import DeepFace
import time

# Cargar las imágenes de superposición
lipstick_image = cv2.imread('../P5/images/labiosrojos.png', cv2.IMREAD_UNCHANGED)
mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)

# Verificar si las imágenes se han cargado correctamente
if lipstick_image is None:
    print("Error: No se pudo cargar la imagen de labios.")
if mustache_image is None:
    print("Error: No se pudo cargar la imagen de bigote.")

# Convertir las imágenes a BGRA si no tienen canal alfa
if lipstick_image is not None and lipstick_image.shape[2] == 3:
    lipstick_image = cv2.cvtColor(lipstick_image, cv2.COLOR_BGR2BGRA)
if mustache_image is not None and mustache_image.shape[2] == 3:
    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)

def overlay_image(background, overlay, position):
    x, y = position
    h, w = overlay.shape[:2]
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay
    for c in range(0, 3):  # Para cada canal de color
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

# Iniciar la cámara
vid = cv2.VideoCapture(0)

last_detection_time = 0  # Almacenamos el tiempo de la última detección
detection_interval = 2.5  # Intervalo de tiempo en segundos para detección de género
gender = None  # Inicializamos la variable de género

while True:
    ret, frame = vid.read()
    if not ret:
        break

    current_time = time.time()

    # realizar el análisis de género cada 2.5 segundos
    if current_time - last_detection_time > detection_interval:
        try:
            # Análisis de género
            obj = DeepFace.analyze(frame, actions=['gender'], enforce_detection=False)
            gender = obj[0]['dominant_gender']
            last_detection_time = current_time  # Actualizamos el tiempo de la última detección
        except Exception as e:
            print("Error de análisis:", e)
            continue

    # Detección de rostros
    faces = DeepFace.extract_faces(frame, detector_backend="retinaface", align=False)
    if len(faces) > 0 and 'facial_area' in faces[0]:
        facial_area = faces[0]['facial_area']
        x, y, w, h = facial_area['x'], facial_area['y'], facial_area['w'], facial_area['h']

        # Aplicar el filtro correspondiente según el género detectado
        if gender == 'Man' and lipstick_image is not None:
            overlay_img = cv2.resize(lipstick_image, (w, h // 2))
            frame = overlay_image(frame, overlay_img, (x, y + h // 2))

        elif gender == 'Woman' and mustache_image is not None:
            overlay_img = cv2.resize(mustache_image, (w, h // 4))
            frame = overlay_image(frame, overlay_img, (x, y + h // 2))

    # Muestra el frame con la superposición
    cv2.imshow("Filtro de Género", frame)

    # Esc para salir
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cerrar cámara y ventanas
vid.release()
cv2.destroyAllWindows()


VERSIÓN CON HAAR CASCADE

In [2]:
import cv2
import time
from deepface import DeepFace

# Cargar las imágenes de superposición
lipstick_image = cv2.imread('../P5/images/labiosrojos.png', cv2.IMREAD_UNCHANGED)
mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)

# Verificar si las imágenes se han cargado correctamente
if lipstick_image is None:
    print("Error: No se pudo cargar la imagen de labios.")
if mustache_image is None:
    print("Error: No se pudo cargar la imagen de bigote.")

# Convertir las imágenes a BGRA si no tienen canal alfa
if lipstick_image is not None and lipstick_image.shape[2] == 3:
    lipstick_image = cv2.cvtColor(lipstick_image, cv2.COLOR_BGR2BGRA)
if mustache_image is not None and mustache_image.shape[2] == 3:
    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)

def overlay_image(background, overlay, position):
    """Superponer la imagen overlay sobre la imagen background en la posición indicada."""
    x, y = position
    h, w = overlay.shape[:2]
    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay

    for c in range(0, 3):  # Para cada canal de color
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

# Iniciar la cámara
vid = cv2.VideoCapture(0)

# Inicializar variables para el análisis de género
gender = None
last_detection_time = 0
detection_interval = 1.5  # Intervalo en segundos para detección de género

while True:
    ret, frame = vid.read()
    if not ret:
        break

    current_time = time.time()
    
    # Solo analizar el género cada 'detection_interval' segundos
    if current_time - last_detection_time > detection_interval or gender is None:
        try:
            # Analizamos el género en una versión más pequeña del frame para acelerar el procesamiento
            resized_frame = cv2.resize(frame, (640, 480))
            obj = DeepFace.analyze(resized_frame, actions=['gender'], enforce_detection=False)
            gender = obj[0]['dominant_gender']
            last_detection_time = current_time
        except Exception as e:
            print("Error de análisis:", e)
            continue

    # Usar Haar Cascade de OpenCV para detectar rostros rápidamente
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")
    faces = face_cascade.detectMultiScale(gray, 1.1, 4)

    for (x, y, w, h) in faces:
        # Aplicar el filtro correspondiente según el género detectado
        if gender == 'Man' and lipstick_image is not None:
            # Reducimos un poco el tamaño de los labios y bajamos su posición
            overlay_img = cv2.resize(lipstick_image, (w // 2, h // 5))
            frame = overlay_image(frame, overlay_img, (x + w // 4, y + int(h * 0.7)))
        
        elif gender == 'Woman' and mustache_image is not None:
            # Ajustamos el tamaño y bajamos un poco el bigote
            overlay_img = cv2.resize(mustache_image, (w // 2, h // 6))
            frame = overlay_image(frame, overlay_img, (x + w // 4, y + int(h * 0.65)))

    # Mostrar el frame con la superposición
    cv2.imshow("Filtro de Genero", frame)

    # Salir con 'ESC'
    if cv2.waitKey(1) & 0xFF == 27:
        break

# Cerrar cámara y ventanas
vid.release()
cv2.destroyAllWindows()


VERSIÓN CON MEDIAPIPE

In [3]:
import cv2
import time
from deepface import DeepFace
import mediapipe as mp
import numpy as np

# Cargar imagen de bigote
mustache_image = cv2.imread('../P5/images/mostacho.png', cv2.IMREAD_UNCHANGED)
if mustache_image is None:
    print("Error: No se pudo cargar la imagen de bigote.")

if mustache_image is not None and mustache_image.shape[2] == 3:
    mustache_image = cv2.cvtColor(mustache_image, cv2.COLOR_BGR2BGRA)

# Función para superponer la imagen del bigote
def overlay_image(background, overlay, position):
    x, y = position
    h, w = overlay.shape[:2]
    bh, bw = background.shape[:2]

    # Verificar límites
    if x >= bw or y >= bh:
        return background  # Si está completamente fuera de la imagen, no hacer nada

    # Ajustar la región si se sale de los límites
    w = min(w, bw - x)
    h = min(h, bh - y)
    overlay = overlay[:h, :w]

    alpha_overlay = overlay[:, :, 3] / 255.0
    alpha_background = 1.0 - alpha_overlay

    for c in range(0, 3):
        background[y:y+h, x:x+w, c] = (alpha_overlay * overlay[:, :, c] +
                                       alpha_background * background[y:y+h, x:x+w, c])
    return background

# Configuración de Mediapipe para detección y mapeo de landmarks faciales
mp_face_mesh = mp.solutions.face_mesh
mp_drawing = mp.solutions.drawing_utils

# Captura de video
vid = cv2.VideoCapture(0)

gender = None
last_detection_time = 0
detection_interval = 2

# Iniciar el modelo de detección de rostros y landmarks
with mp_face_mesh.FaceMesh(min_detection_confidence=0.5, min_tracking_confidence=0.5) as face_mesh:
    while True:
        ret, frame = vid.read()
        if not ret:
            break

        # Detectar género a intervalos
        current_time = time.time()
        if current_time - last_detection_time > detection_interval or gender is None:
            try:
                resized_frame = cv2.resize(frame, (640, 480))
                obj = DeepFace.analyze(resized_frame, actions=['gender'], enforce_detection=False)
                gender = obj[0]['dominant_gender']
                last_detection_time = current_time
            except Exception as e:
                print("Error de análisis:", e)
                continue

        # Procesar landmarks faciales con Mediapipe
        frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_mesh.process(frame_rgb)

        if results.multi_face_landmarks:
            for face_landmarks in results.multi_face_landmarks:
                ih, iw, _ = frame.shape

                if gender == 'Man':
                    # Puntos alrededor de ambos labios (superior e inferior)
                    lip_points = [
                        61, 146, 91, 181, 84, 17, 314, 405, 321, 375, 291, 308, 
                        324, 318, 402, 317, 14, 87, 178,  95, 88, 178, 191, 80,
                        81, 82, 13, 312, 311, 310, 415, 308
                    ]
                    lip_coords = [(int(face_landmarks.landmark[point].x * iw), int(face_landmarks.landmark[point].y * ih)) for point in lip_points]

                    # Crear una máscara roja sobre los labios
                    lip_mask = np.zeros_like(frame)
                    cv2.fillPoly(lip_mask, [np.array(lip_coords, np.int32)], (0, 0, 255))
                    frame = cv2.addWeighted(frame, 1, lip_mask, 0.4, 0)

                elif gender == 'Woman' and mustache_image is not None:
                    # Obtener puntos clave para colocar el bigote
                    left_mouth_corner = face_landmarks.landmark[78]
                    right_mouth_corner = face_landmarks.landmark[308]

                    # Calcular posición y tamaño del bigote
                    x1, y1 = int(left_mouth_corner.x * iw), int(left_mouth_corner.y * ih)
                    x2, y2 = int(right_mouth_corner.x * iw), int(right_mouth_corner.y * ih)

                    # Calcular ancho del bigote y posicionarlo
                    mustache_width = x2 - x1
                    mustache_height = int(mustache_width * mustache_image.shape[0] / mustache_image.shape[1])
                    overlay_img = cv2.resize(mustache_image, (mustache_width, mustache_height))

                    # Posicionar bigote ligeramente por encima del centro de la boca
                    y_offset = int(y1 - mustache_height / 2)
                    frame = overlay_image(frame, overlay_img, (x1, y_offset))

        # Mostrar el video con los filtros aplicados
        cv2.imshow("Filtro de Genero", frame)

        if cv2.waitKey(1) & 0xFF == 27:
            break

vid.release()
cv2.destroyAllWindows()
